In [1]:
import time
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('pandas udf').getOrCreate()

In [3]:
def plus(a,b):
 return a+b

In [4]:
spark.udf.register("plus",plus)

<function __main__.plus(a, b)>

In [7]:
start = time.time()
df = spark.range(100000000).selectExpr("id AS a","id AS b")
df.selectExpr("count(plus(a,b))").collect()
end = time.time()-start
print(f"Time taken {end}")

Time taken 62.69094705581665


In [8]:
df.selectExpr("count(plus(a,b))").explain()

== Physical Plan ==
*(3) HashAggregate(keys=[], functions=[count(pythonUDF0#43)])
+- Exchange SinglePartition, true, [id=#132]
   +- *(2) HashAggregate(keys=[], functions=[partial_count(pythonUDF0#43)])
      +- *(2) Project [pythonUDF0#43]
         +- BatchEvalPython [plus(a#28L, b#29L)], [pythonUDF0#43]
            +- *(1) Project [id#26L AS a#28L, id#26L AS b#29L]
               +- *(1) Range (0, 100000000, step=1, splits=12)




In [11]:
from pyspark.sql.types import LongType
from pyspark.sql.functions import pandas_udf,count

@pandas_udf(LongType())
def plus_pandas(a,b):
 return a+b

In [13]:
spark.udf.register("plus_pandas",plus_pandas)

<function __main__.plus_pandas(a, b)>

In [14]:
start = time.time()
df = spark.range(100000000).selectExpr("id AS a","id AS b")
df.selectExpr("count(plus_pandas(a,b))").collect()
end = time.time()-start
print(f"Time taken {end}")

Time taken 14.810265064239502


In [15]:
df.selectExpr("count(plus_pandas(a,b))").explain()

== Physical Plan ==
*(3) HashAggregate(keys=[], functions=[count(pythonUDF0#69L)])
+- Exchange SinglePartition, true, [id=#200]
   +- *(2) HashAggregate(keys=[], functions=[partial_count(pythonUDF0#69L)])
      +- *(2) Project [pythonUDF0#69L]
         +- ArrowEvalPython [plus_pandas(a#54L, b#55L)], [pythonUDF0#69L], 200
            +- *(1) Project [id#52L AS a#54L, id#52L AS b#55L]
               +- *(1) Range (0, 100000000, step=1, splits=12)


